In [26]:
import json
import time
import re
from typing import Dict, Any, Optional

from dotenv import load_dotenv
import os

from openai import OpenAI

In [27]:
MODEL = "gpt-5"
REQUESTS_PER_MINUTE = 1            # set lower if you hit rate limits
RETRY_LIMIT = 3
SLEEP_BETWEEN_CALLS = 60.0 / REQUESTS_PER_MINUTE

# Master list of European countries (geographic Europe + microstates + Kosovo; Turkey & Azerbaijan included as transcontinental)
EUROPEAN_COUNTRIES = [
    "Albania", "Andorra", "Armenia", "Austria", "Azerbaijan", "Belarus", "Belgium",
    "Bosnia and Herzegovina", "Bulgaria", "Croatia", "Cyprus", "Czechia", "Denmark",
    "Estonia", "Finland", "France", "Georgia", "Germany", "Greece", "Hungary",
    "Iceland", "Ireland", "Italy", "Kosovo", "Latvia", "Lithuania",
    "Luxembourg", "Malta", "Moldova", "Montenegro", "Netherlands",
    "North Macedonia", "Norway", "Poland", "Portugal", "Romania", "Russia",
    "San Marino", "Serbia", "Slovakia", "Slovenia", "Spain", "Sweden",
    "Switzerland", "Ukraine", "United Kingdom"
]

In [28]:
def extract_json(text: str) -> Optional[Dict[str, Any]]:
    """
    Try to parse a small JSON object from a model response.
    Accepts a bare JSON object or JSON fenced in code blocks or extra prose.
    """
    # Look for the first JSON object in the text with a simple brace counter
    # as a fallback if standard parsing fails.
    # First, try straightforward parse:
    try:
        return json.loads(text)
    except Exception:
        pass

    # Try to extract from a code block
    codeblock = re.search(r"```(?:json)?\s*({.*?})\s*```", text, flags=re.DOTALL | re.IGNORECASE)
    if codeblock:
        try:
            return json.loads(codeblock.group(1))
        except Exception:
            pass

    # Braces sweep
    start_idxs = [m.start() for m in re.finditer(r"\{", text)]
    for start in start_idxs:
        depth = 0
        for i in range(start, len(text)):
            if text[i] == "{":
                depth += 1
            elif text[i] == "}":
                depth -= 1
                if depth == 0:
                    candidate = text[start:i+1]
                    try:
                        return json.loads(candidate)
                    except Exception:
                        break
    return None

In [29]:
def ask_country_privatisation(client: OpenAI, country: str) -> Dict[str, str]:
    """
    Query GPT-5 about privatisation status for one country.
    Returns {"privatized": "yes|no|partly"}; defaults to {"privatized":"partly"} if uncertain.
    """
    prompt = (
        f"For the European country '{country}', has the long-distance electricity transmission "
        f"network (the high-voltage TSO grid) been privatized? "
        f"Conclude your answer with a valid, minified JSON in this exact schema:\n"
        f'{{"privatized":"yes"|"no"|"partly"}}.\n'
        f"Use the web_search tool to verify before answering. "
        f"Partly means that while the companies are publicly traded/owned, the government stays in control over operations."
    )

    for attempt in range(1, RETRY_LIMIT + 1):
        try:
            resp = client.responses.create(
                model=MODEL,
                tools=[{"type": "web_search"}],
                input=prompt,
            )
            text = getattr(resp, "output_text", None) or str(resp)
            data = extract_json(text)
            if not data or "privatized" not in data:
                raise ValueError(f"Could not parse JSON for {country}: {text!r}")
            val = str(data["privatized"]).strip().lower()
            if val not in {"yes", "no", "partly"}:
                # Normalize some common variants
                if val in {"partial", "mixed", "semi", "semi-privatized", "semi privatized"}:
                    val = "partly"
                elif val in {"state-owned", "state owned", "public"}:
                    val = "no"
                elif val in {"privatized", "private"}:
                    val = "yes"
                else:
                    val = "partly"
            return {"privatized": val}
        except Exception as e:
            print(f"Error on attempt {attempt} for {country}: {e}")
            if attempt >= RETRY_LIMIT:
                # Fall back to 'partly' on repeated failures
                return {"privatized": None}
            time.sleep(SLEEP_BETWEEN_CALLS)

In [ ]:
from tqdm import tqdm

load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)
results: Dict[str, Dict[str, str]] = {}

for country in tqdm(EUROPEAN_COUNTRIES, desc="Processing countries"):
    result = ask_country_privatisation(client, country)
    results[country] = result
    time.sleep(SLEEP_BETWEEN_CALLS)

# Print the final structured JSON
with open("privatisation_results.json", "w", encoding="utf-8") as f:
    json.dump(results, f, ensure_ascii=False, separators=(",", ":"))
print("Results saved to privatisation_results.json")

Processing countries:   0%|          | 0/46 [00:00<?, ?it/s]

Albania: {'privatized': 'no'}


Processing countries:   2%|▏         | 1/46 [01:23<1:02:45, 83.68s/it]

Andorra: {'privatized': 'no'}


Processing countries:   4%|▍         | 2/46 [03:04<1:08:46, 93.77s/it]

...

Ukraine: {'privatized': 'no'}


Processing countries:  98%|█████████▊| 45/46 [1:18:40<01:40, 100.58s/it]

United Kingdom: {'privatized': 'partly'}


Processing countries: 100%|██████████| 46/46 [1:21:14<00:00, 105.97s/it]

Results saved to privatisation_results.json
